In [ ]:
! pip install -Uq torch transformers langchain langchain-community chromadb pypdf sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 121.2 MB/s e

In [ ]:
import os
import torch
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, pipeline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
pdf_dir = "docs"
loader = PyPDFDirectoryLoader(pdf_dir)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " "]
)
chunks = text_splitter.split_documents(documents)

In [ ]:
len(chunks)

364

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
)

In [ ]:
db_dir = "db"
if os.path.exists(db_dir):
    print("📁 Chargement de la base existante...")
    vectorstore = Chroma(persist_directory=db_dir, embedding_function=embeddings)
else:
    print("🆕 Création de la base vectorielle...")
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=db_dir
    )
    vectorstore.persist()

🆕 Création de la base vectorielle...


In [ ]:

# Prompt généré par claude
prompt_template = """Tu es un assistant médical expert. Utilise UNIQUEMENT les informations du contexte fourni.

CONTEXTE MÉDICAL:
{context}

RÈGLES STRICTES:
1. Réponds UNIQUEMENT avec les informations du contexte
2. Si l'information n'est pas dans le contexte, dis exactement: "Cette information n'est pas disponible dans les documents fournis."
3. Utilise le vocabulaire médical précis du contexte
4. Cite les passages pertinents entre guillemets
5. Réponds en français médical professionnel

QUESTION: {question}

RÉPONSE MÉDICALE:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

text_pipeline = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.3",
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.3,
    top_p=0.9,
    repetition_penalty=1.2,
    device=0 if torch.cuda.is_available() else -1,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=text_pipeline)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

In [ ]:
def ask_question(question):
    """Fonction pour poser une question au système"""
    print(f"\n❓ Question: {question}")
    print("🔍 Recherche en cours...")

    try:
        result = qa_chain({"query": question})
        answer = result["result"]
        sources = result.get("source_documents", [])

        print(f"💡 Réponse: {answer}")

        if sources:
            print(f"\n📚 Sources trouvées ({len(sources)} documents):")
            for i, doc in enumerate(sources[:2], 1):
                preview = doc.page_content
                print(f"  {i}. {preview}")

        return answer

    except Exception as e:
        error_msg = f"❌ Erreur: {str(e)}"
        print(error_msg)
        return error_msg

In [ ]:
# Questions de test
test_questions = [
    "Comment sont classifiées les AHAI ?",
    "Qui a coordonné le PNDS ?",
    "Quelles sont les sources utilisées pour le guide ?",
    "Quels sont les traitements mentionnés ?",
    "Qu'est-ce que le PNDS ?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*20} TEST {i} {'='*20}")
    ask_question(question)


==================== TEST 1 ====================

❓ Question: Comment sont classifiées les AHAI ?
🔍 Recherche en cours...
💡 Réponse:  Les AHAI sont classifiées selon les propriétés immunochimiques de l'auto-anticorps en cause. On distingue principalement les AHAI à auto-anticorps « chauds ».

📚 Sources trouvées (3 documents):
  1. 17 
II. Synthèse à l’intention des médecins spécialistes  
(Médecine Interne et Immunologie clinique, Hématologie) 
susceptibles de prendre en charge un patient atteint 
d’AHAI 
A) Précisions concernant la classification des AHAI 
 
On distingue essentiellement selon les propriétés immuno -chimiques de l’auto -
anticorps en cause :  
 
Les anémies hémolytiques auto -immunes (AHAI) à auto -Ac « chauds », 
anticorps qui exercent leur activité hémolytique maximale à des températures
  2. 18 
Tableau 3 
 
Classification et caractéristiques principales des AHAI. 
 
 
Type d’AHAI 
 
 
Terrain / Clinique 
 
Formes 
« secondaires »  
 
Classe d’Ig. 
 
Agglutinines 


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


💡 Réponse:  Les traitements mentionnés sont la corticoïthérapie [corticoids], la rituximab, les immunosuppresseurs, la splénectomie ainsi que le mTOR inhibiteur.

📚 Sources trouvées (3 documents):
  1. results in 73 patients. Am J Hematol. 1993 Dec;44(4):237–42. 
 
 Traitements ± leurs complications (articles originaux) des AHAI 
à Ac. chauds ( corticoides, rituximab, immuno suppresseurs, 
Splénectomie). 
 
Sorin B , Fadlallah J, Garzaro M, Vigneron J, Bertinchamp R, Boutboul D, 
Oksenhendler E, Fieschi C, Malphettes M, Galicier L.  Real-life use of mTOR 
inhibitor-based therapy in adults with autoimmune cytopenia highlights strong
  2. mesures habituelles associées à la corticothérapie (règles hygiéno -diététiques, 
prévention de l’ostéoporose…) doivent bien sûr être entreprises et le traitement ne 
doit pas être arrêté de façon brutale.

==================== TEST 5 ====================

❓ Question: Qu'est-ce que le PNDS ?
🔍 Recherche en cours...
💡 Réponse:  Le PNDS est un protocole 